In [ ]:
# === STEP 1: LOAD THE NEW OUTBOUND DATA ===
# === INITIAL SETUP: RUN THIS CELL FIRST ===
import pandas as pd
import numpy as np

# Define the path to your new Outbound file
outbound_file_path = r"C:\Users\erictr\Documents\Python Billing Project\Outbound Data\CKL Order Allocation Report.xlsx" # <<< CHANGE THIS TO YOUR ACTUAL FILE PATH

# Read the new file into a DataFrame called 'outbound_df'
outbound_df = pd.read_excel(outbound_file_path)

# Let's see what we're working with
print("✅ Outbound Data Loaded Successfully!")
print(f"Shape: {outbound_df.shape} (rows, columns)")

In [ ]:
# Add new columns with default values
outbound_df['Picking Count'] = 0
outbound_df['Denom'] = 1
outbound_df['Pick Type'] = ''
outbound_df['Full_Picking Count'] = 0
outbound_df['Partial_Picking Count'] = 0

In [ ]:
import pandas as pd
import numpy as np

# === PART 1: Update 'Picking Count' for UOM == 'CTN' ===
# This sets the value for rows where UOM is 'CTN'
outbound_df.loc[outbound_df['UOM'] == 'CTN', 'Picking Count'] = outbound_df.loc[outbound_df['UOM'] == 'CTN', 'Allocated']
print("✅ Step 1 Complete: 'Picking Count' updated for UOM = 'CTN'")

# === PART 2: OVERRIDE 'Picking Count' for specific Articles ===
# These rules will OVERWRITE the values for these specific articles,
# even if they were set by the UOM rule above.

# Now apply each rule one by one using .loc
outbound_df.loc[outbound_df['Article'] == 'ST-C730000002', 'Picking Count'] = outbound_df['Allocated'] / 10
outbound_df.loc[outbound_df['Article'] == 'ST-C230000165', 'Picking Count'] = np.nan
outbound_df.loc[outbound_df['Article'] == 'ST-C730000093', 'Picking Count'] = outbound_df['Allocated'] / 5
outbound_df.loc[outbound_df['Article'] == 'ST-C730000023', 'Picking Count'] = outbound_df['Allocated'] / 5
outbound_df.loc[outbound_df['Article'] == 'ST-C730000226', 'Picking Count'] = outbound_df['Allocated']
outbound_df.loc[outbound_df['Article'] == 'ST-C730000021', 'Picking Count'] = outbound_df['Allocated'] / 10
outbound_df.loc[outbound_df['Article'] == 'ST-C230000534', 'Picking Count'] = np.nan

print("✅ Step 2 Complete: 'Picking Count' updated based on Article rules.")

# === FINAL CHECK: Print a sample to verify the column ===
print("\n🔍 Final Check - Sample of relevant columns:")
print(outbound_df[['Article', 'UOM', 'Allocated', 'Picking Count']].head(15))

In [ ]:
# === STEP 1: LOAD THE DENOM MASTER FILE ===

# Define the path to your Denom Master file
denom_master_path = r"C:\Users\erictr\Documents\Python Billing Project\CF Max Pallet Qty(Denom)_01072025.xlsx" # <<< CHANGE THIS TO YOUR ACTUAL FILE PATH

# Read the master file into a DataFrame
denom_master_df = pd.read_excel(denom_master_path)

# Let's see what we're working with
print("✅ Denom Master Data Loaded Successfully!")
print(f"Shape: {denom_master_df.shape} (rows, columns)")
print("\nFirst 5 rows of Denom Master:")
print(denom_master_df.head().to_string())

In [ ]:
# === STEP 2 (CORRECTED): UPDATE EXISTING 'Denom' COLUMN ===

# Use the EXACT column name, including the space at the end: 'Max Pallet Qty '
denom_mapping = denom_master_df.set_index('Article')['Max Pallet Qty ']

# Use the .map() function to update the 'Denom' column
outbound_df['Denom'] = outbound_df['Article'].map(denom_mapping)

print("✅ Update complete! 'Denom' column updated with values from master file.")
print(outbound_df[['Article', 'Denom']].head(15))

In [ ]:
# === STEP: CONVERT 'Denom' COLUMN TO NUMBERS ===

# Check the current data type
print("Current data type of 'Denom':", outbound_df['Denom'].dtype)

# Convert the column to numeric. Errors will be converted to NaN (so text becomes NaN)
outbound_df['Denom'] = pd.to_numeric(outbound_df['Denom'], errors='coerce')

# Check the new data type
print("New data type of 'Denom':", outbound_df['Denom'].dtype)

# Check if we have any non-numeric values that were forced to NaN
print("Number of NaN values in 'Denom' after conversion:", outbound_df['Denom'].isna().sum())

In [ ]:
# === CALCULATE 'Pick Type' BASED ON EXCEL LOGIC ===

conditions = [
    outbound_df['Denom'].isna(),                           # ISNA(Q2) - If Denom is NaN/blank
    outbound_df['Picking Count'] >= outbound_df['Denom']   # I2 >= Q2 - If Picking Count >= Denom
]

choices = [
    'Partial',  # Result if Denom is NaN
    'Full'      # Result if Picking Count >= Denom
]

# Apply the conditions: default value is 'Partial' for any case not caught above
outbound_df['Pick Type'] = np.select(conditions, choices, default='Partial')

print("✅ 'Pick Type' column updated based on Excel logic!")
print(outbound_df[['Article', 'Picking Count', 'Denom', 'Pick Type']].head(15))

In [ ]:
# === UPDATE 'Full_Picking Count' COLUMN ===

# If 'Pick Type' is 'Full', set 'Full_Picking Count' to 1, otherwise keep it 0
outbound_df.loc[outbound_df['Pick Type'] == 'Full', 'Full_Picking Count'] = 1

print("✅ 'Full_Picking Count' column updated!")
print(outbound_df[['Article', 'Pick Type', 'Full_Picking Count']].head(15))

In [ ]:
# === UPDATE 'Partial_Picking Count' COLUMN ===

# First, convert the column to float to avoid dtype conflict
outbound_df['Partial_Picking Count'] = outbound_df['Partial_Picking Count'].astype(float)

# If 'Pick Type' is 'Partial', copy value from 'Allocated' to 'Partial_Picking Count'
outbound_df.loc[outbound_df['Pick Type'] == 'Partial', 'Partial_Picking Count'] = outbound_df.loc[outbound_df['Pick Type'] == 'Partial', 'Picking Count']

print("✅ 'Partial_Picking Count' column updated!")
print(outbound_df[['Article', 'Pick Type', 'Allocated', 'Partial_Picking Count']].head(15))

In [ ]:
# 1. Define the path for the NEW file
new_file_path = r"C:\Users\erictr\Documents\Python Billing Project\Outbound Data\Result\Modified_Outbound_Data.xlsx"

# 2. List the 5 columns you want to style
columns_to_style = ['Picking Count', 'Denom', 'Pick Type', 'Full_Picking Count', 'Partial_Picking Count']

# 3. First export the DataFrame to Excel
outbound_df.to_excel(new_file_path, index=False)

# 4. Now use openpyxl to apply formatting to the HEADERS only
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill

# Load the workbook we just created
wb = load_workbook(new_file_path)
ws = wb.active

# Define the light yellow color for headers
light_yellow_fill = PatternFill(start_color='FFFFD6', end_color='FFFFD6', fill_type='solid')
header_font = Font(name='Calibri', size=11, color='800000', bold=True)  # Black Calibri, bold

# Find and format ONLY the header cells for our target columns
for col_idx, cell in enumerate(ws[1], 1):  # Look at header row (row 1)
    if cell.value in columns_to_style:
        # Format only this header cell
        cell.fill = light_yellow_fill
        cell.font = header_font

# Save the formatted workbook
wb.save(new_file_path)

print(f"✅ SUCCESS! Formatted file saved to: {new_file_path}")
print(f"✅ Applied light yellow background to HEADERS of: {columns_to_style}")